In [1]:
import pandas as pd
import numpy as np

In [3]:
# uploading file
News = pd.read_csv(r'C:\Users\Ania Sikora\Desktop\Python - zaawansowane\dfet\sentymenty\tytuły i daty\news_reuters.csv', names=["Index", "Company", "Date", "Title", "Article", "Category"])

In [4]:
# these columns are not necessary
News = News.drop(['Article', 'Category', 'Company'], axis=1)

In [5]:
News

,Index,Date,Title
0,AA,20110707,Alcoa profit seen higher on aluminum price surge
1,AA,20110708,Global markets weekahead: Lacking conviction
2,AA,20110708,Jobs halt Wall Street rally investors eye ear...
3,AA,20110708,REFILE-TABLE-Australia's top carbon polluters
4,AA,20110708,US STOCKS-Jobs data hits stocks but earnings ...
...,...,...,...
215283,ZOES,20140411,Zoe's Kitchen shares jump more than 70 pct in ...
215284,ZOES,20140411,Zoe's Kitchen shares jump more than 70 percent...
215285,ZX,20170217,BRIEF-China Zenix Auto International announces...
215286,ZYNE,20170118,BRIEF-Zynerba Pharmaceuticals announces propos...


In [6]:
# choosing only titles concerning each of 3 companies
mask1 = News['Index'].str.contains('MSFT',na=False)
mask2 = News['Index'].str.contains('AAPL',na=False)
mask3 = News['Index'].str.contains('BA',na=False)
Microsoft = News[mask1]  
Apple = News[mask2]
Boeing = News[mask3]

In [7]:
Microsoft

,Index,Date,Title
144549,MSFT,20110706,Facebook launches video chat with Skype
144550,MSFT,20110706,UPDATE 5-Facebook launches video chat with Skype
144551,MSFT,20110707,Buffett donates $1.5 billion to Gates Foundation
144552,MSFT,20110707,Canada eyes Nortel patent sale review- minister
144553,MSFT,20110707,Facebook launches video chat with Skype
...,...,...,...
147218,MSFT,20161213,Factbox: Trump meetings include rapper Kanye W...
147219,MSFT,20161213,Microsoft co-founder Gates Trump discuss inno...
147220,MSFT,20161220,Microsoft wins $927 million Pentagon contract:...
147221,MSFT,20170113,BlackRock CEO says Trump 'tax holiday' may do ...


In [8]:
# column Index won't be usefull anymore
Microsoft = Microsoft.drop(['Index'], axis=1)
Apple = Apple.drop(['Index'], axis=1)
Boeing = Boeing.drop(['Index'], axis=1)

In [9]:
# moving titles from the same day to one row
def change_format(firm):
    Microsoft_out = firm.set_index(['Date',firm.groupby(['Date']).cumcount()+1]).unstack().sort_index(level=1, axis=1)
    Microsoft_out.columns = Microsoft_out.columns.map('{0[0]}_{0[1]}'.format)
    Microsoft_out.reset_index()
    Microsoft_out = Microsoft_out.replace(np.nan, ' ')
    return Microsoft_out



In [10]:
#applying our function on 3 dataframes
df_MS = change_format(Microsoft)
df_AP = change_format(Apple)
df_BA = change_format(Boeing)

In [11]:
df_MS

,Title_1,Title_2,Title_3,Title_4,Title_5,Title_6,Title_7,Title_8,Title_9,Title_10
Date,,,,,,,,,,
20110706,Facebook launches video chat with Skype,UPDATE 5-Facebook launches video chat with Skype,,,,,,,,
20110707,Buffett donates $1.5 billion to Gates Foundation,Canada eyes Nortel patent sale review- minister,Facebook launches video chat with Skype,Hulu owners seeking best sale value says Disn...,UPDATE 1-Buffett donates $1.5 billion to Gates...,UPDATE 1-Canada eyes Nortel patent sale review...,,,,
20110708,Buffett gives $1.78 billion to Gates family c...,IFR-US CREDIT-Bondholders take backseat to sha...,Local.com shares leap on Google deal,UPDATE 2-Buffett gives $1.78 bln to Gates fam...,,,,,,
20110709,Murdoch: Closing newspaper a 'collective decis...,,,,,,,,,
20110711,Bondholders take backseat to shareholders,Courts OK Nortel patent sale to Apple/RIM group,UPDATE 3-Courts OK Nortel patent sale to Apple...,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
20161205,Web giants to cooperate on removal of extremis...,,,,,,,,,
20161206,Microsoft gains EU approval for $26 billion bu...,,,,,,,,,
20161213,Factbox: Trump meetings include rapper Kanye W...,Microsoft co-founder Gates Trump discuss inno...,,,,,,,,


In [12]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# create function for sentiment analysis
analyser = SentimentIntensityAnalyzer()
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)['compound']
    return(score)

In [13]:
def sentiment(company):
    cols = []
    for i in range(1, len(company.columns)+1):
        # list to store the sentiment values
        col = ("Title_{}".format(i))
        cols.append(col)

    for col in (cols):
        # to a string
        company[col]= company[col].apply(lambda x: str(x))
    for col in (cols):
        # applying sentiment analysis
        company[col]= company[col].apply(lambda x: sentiment_analyzer_scores(x))
    company.reset_index(inplace=True)
    # caluculating mean of all sentiment scores from all titles from each day
    weighted_comp = []
    for i in range(0,len(company)):
        a = company.loc[i,cols].tolist()
        n = 0
        i= 0 
        length = len(a)  
        while(i<length):
            if(a[i]==n):
                a.remove(a[i])
                length = length -1  
                continue
            i = i+1
        weighted_comp.append(np.average(a))
#adding a column with the mean of  all sentiment scores from all titles from each day
    company['compound_mean'] = weighted_comp
    company = company.drop(cols, axis=1)
    company.fillna(company['compound_mean'].mean(), inplace = True)
    company['Date'] = pd.to_datetime(company['Date'], format='%Y%m%d')
    return company

In [14]:
#applying Sentiment Analaysis on our 3 dataframes
df_BA = sentiment(df_BA)
df_MS = sentiment(df_MS)
df_AP = sentiment(df_AP)

C:\Users\Ania Sikora\Anaconda3\lib\site-packages\numpy\lib\function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\Ania Sikora\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [15]:
df_BA

,Date,compound_mean
0,2011-07-06,-0.493900
1,2011-07-07,0.074200
2,2011-07-08,0.084700
3,2011-07-09,0.153100
4,2011-07-10,0.055833
...,...,...
1528,2017-02-06,0.055833
1529,2017-02-07,-0.273000
1530,2017-02-10,0.055833
1531,2017-02-13,-0.361200


In [16]:
#saving the dataframes 
df_BA.to_csv('df_BA.csv')

In [17]:
df_MS.to_csv('df_MS.csv')

In [18]:
df_AP.to_csv('df_AP.csv')